**Torchvision + BERT**

In [2]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import pandas as pd
import os

# Load the dataset

df = pd.read_csv('/content/drive/MyDrive/cleaned_dataset.csv')
print(df.shape)
print(df['Review Image'][0])

# Function to update the image path
def update_image_path(old_path):
    # Split the path using os.path.basename, handling both Windows and Unix-like paths
    filename = os.path.basename(old_path.replace('\\', '/'))  # Handle Windows backslashes
    # Construct the new path (update this to your new directory path)
    new_path = f'/content/drive/MyDrive/hotel images/{filename}'
    return new_path

# Apply the function to the 'Review Image' column
df['Review Image'] = df['Review Image'].apply(update_image_path)

# Save the updated dataset
df.to_csv('/content/drive/MyDrive/updated_cleaneddataset.csv', index=False)

print("Image paths updated successfully.")
print(df['Review Image'][0])


(1678, 4)
E:\OneDrive\Desktop\sem7\BTP\project_dataset\Hotel_images\Hotel_images\hotel images\d1_1.jpeg
Image paths updated successfully.
/content/drive/MyDrive/hotel images/d1_1.jpeg


In [5]:
import pandas as pd
import os

# Load the dataset

df = pd.read_csv('/content/drive/MyDrive/updated_cleaneddataset.csv')
print(df.shape)
print(df['Review Image'][0])

(1678, 4)
/content/drive/MyDrive/hotel images/d1_1.jpeg


**5-Fold Cross Validation**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from transformers import BertTokenizer, BertModel
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader

# Define the combined model
class CombinedModel(nn.Module):
    def __init__(self, text_dim, image_dim, combined_dim):
        super(CombinedModel, self).__init__()
        self.text_fc = nn.Linear(text_dim, combined_dim)
        self.image_fc = nn.Linear(image_dim, combined_dim)
        self.fc = nn.Linear(combined_dim * 2, 5)  # Output layer for 5 classes

    def forward(self, text_features, image_features):
        text_out = self.text_fc(text_features)
        image_out = self.image_fc(image_features)
        combined = torch.cat((text_out, image_out), dim=1)
        output = self.fc(combined)
        return output

# Define the custom dataset class
class ReviewDataset(Dataset):
    def __init__(self, texts, images, labels, tokenizer, transform=None):
        self.texts = texts
        self.images = images
        self.labels = labels
        self.tokenizer = tokenizer
        self.transform = transform
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')
        self.image_feature_size = 150528  # 224x224x3 (assuming image features)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Process text
        text = self.texts[idx]
        if not isinstance(text, str):
            text = str(text)  # Ensure text is a string

        # Tokenize the text input
        inputs = self.tokenizer(text, return_tensors="pt", padding='max_length', truncation=True, max_length=512)
        input_ids = inputs['input_ids'].squeeze(0)
        attention_mask = inputs['attention_mask'].squeeze(0)

        # Extract text features using the BERT model
        with torch.no_grad():
            text_features = self.bert_model(input_ids=input_ids.unsqueeze(0), attention_mask=attention_mask.unsqueeze(0)).last_hidden_state.mean(dim=1)
        text_features = text_features.squeeze(0)

        # Process image
        image_path = self.images[idx]
        try:
            image = Image.open(image_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            image_features = image.view(-1)
        except FileNotFoundError:
            # Handle cases where the image file is missing or invalid
            print(f"Warning: Image not found for index {idx}, using default zero image")
            image_features = torch.zeros(self.image_feature_size)

        # Get the label and convert it to a tensor
        label = torch.tensor(self.labels[idx])

        return text_features, image_features, label

# Define the transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define dimensions
text_dim = 768  # BERT output dimension
image_dim = 150528  # 224x224x3
combined_dim = 256

# Training function with progress percentage
def train_model(train_loader, model, criterion, optimizer, scheduler, device, grad_clip_value):
    model.train()
    total_batches = len(train_loader)
    for batch_idx, (text_features, image_features, labels) in enumerate(train_loader):
        text_features = text_features.to(device)
        image_features = image_features.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(text_features.float(), image_features.float())
        loss = criterion(outputs, labels)
        loss.backward()

        # Apply gradient clipping to avoid instability
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip_value)

        optimizer.step()
        scheduler.step(loss)  # Update learning rate based on loss

        # Calculate progress percentage
        progress = (batch_idx + 1) / total_batches * 100
        print(f"Training Loss: {loss.item():.4f} | Progress: {progress:.2f}%")

# Predict function
def predict(model, dataloader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for text_features, image_features, _ in dataloader:
            text_features = text_features.to(device)
            image_features = image_features.to(device)
            output = model(text_features.float(), image_features.float())
            preds = torch.argmax(output, dim=1)  # Get class indices
            predictions.extend(preds.cpu().tolist())  # Convert to list and extend
    return predictions

# Calculate accuracy function
def calculate_accuracy(predictions, labels):
    correct = sum(1 for pred, actual in zip(predictions, labels) if pred == actual)
    total = len(labels)
    accuracy = correct / total * 100
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy

# Stratified Cross-Validation
def stratified_cross_val(csv_file, tokenizer, transform, device, n_splits=5):
    # Load the dataset
    data = pd.read_csv(csv_file, encoding='ISO-8859-1')
    texts = data['Review Text'].tolist()
    images = data['Review Image'].tolist()
    labels = LabelEncoder().fit_transform(data['Rating'].tolist())

    # Initialize StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Store fold results
    fold_accuracies = []

    # Perform cross-validation
    for fold, (train_idx, test_idx) in enumerate(skf.split(texts, labels)):
        print(f"\nFold {fold+1}/{n_splits}")

        # Split data based on current fold indices
        train_texts = [texts[i] for i in train_idx]
        test_texts = [texts[i] for i in test_idx]
        train_images = [images[i] for i in train_idx]
        test_images = [images[i] for i in test_idx]
        train_labels = [labels[i] for i in train_idx]
        test_labels = [labels[i] for i in test_idx]

        # Create DataLoaders
        train_dataset = ReviewDataset(train_texts, train_images, train_labels, tokenizer, transform)
        test_dataset = ReviewDataset(test_texts, test_images, test_labels, tokenizer, transform)
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)  #  batch size
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)  #  batch size

        # Initialize model, criterion, and optimizer for each fold
        model = CombinedModel(text_dim, image_dim, combined_dim)
        model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.01)

        # Learning rate scheduler (ReduceLR on plateau)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

        # Gradient clipping value
        grad_clip_value = 1.0  # To handle exploding gradients

        # Train the model on the current fold
        train_model(train_loader, model, criterion, optimizer, scheduler, device, grad_clip_value)

        # Evaluate the model
        predictions = predict(model, test_loader, device)
        fold_accuracy = calculate_accuracy(predictions, test_labels)
        fold_accuracies.append(fold_accuracy)

    # Print average accuracy across all folds
    avg_accuracy = sum(fold_accuracies) / len(fold_accuracies)
    print(f"\nAverage Accuracy across {n_splits} folds: {avg_accuracy:.2f}%")

# Example usage
device = torch.device('cuda')  # Specify GPU
stratified_cross_val('/content/drive/MyDrive/updated_cleaneddataset.csv', tokenizer, transform, device)
